In [6]:
import numpy as np
import math
import os

In [7]:
def getParams(stdMethod, sizeRange):
    paramDict = {}
    
    xNum = int(input("Number x nanoparticles: "))
    yNum = int(input("Number y nanoparticles: "))
    zNum = int(input("Number z nanoparticles: "))
    
    if sizeRange == True:
        dString = input("Desired Diameter range (in nm), use ' ' to separate: ")
        dDiameter = [float(i) for i in dString.split()]
        elDensity = float(input("Density of electrons (#/nm^3): "))
    else:
        dDiameter = float(input("Desired Diameter (in nm): "))
        elDensity = float(input("Density of electrons (#/nm^3): "))
    
    if stdMethod == "percent":
        dSTD = float(input("Percent Nanoparticle Size Variation (std): "))
    elif stdMethod == "fixed":
        dSTD = float(input("Nanoparticle Size Standard Deviation (in nm): "))
        
    lSTD = float(input("Nanoparticle Location Standard Deviation (in nm): "))
    vDensity = float(input("Percent Vacancies (in the QDxV(1-x) style): "))
        
    npnumber = xNum*yNum*zNum
    
    paramDict['npnumber'] = npnumber
    paramDict['xNum'] = xNum
    paramDict['yNum'] = yNum
    paramDict['zNum'] = zNum
    paramDict['dDiameter'] = dDiameter
    paramDict['dSTD'] = dSTD
    paramDict['lSTD'] = lSTD
    paramDict['electronDensity'] = elDensity
    paramDict['vacancyDensity'] = vDensity
    
    return paramDict

In [8]:
#the most general lattice, all other bravais lattices can be derived from this one
class triclinic_lattice:
    def __init__(self,stdMethod,sizeRange,currDiameter,paramDict):
        self.npnumber = paramDict['npnumber']
        self.xNum = paramDict['xNum']
        self.yNum = paramDict['yNum']
        self.zNum = paramDict['zNum']
        self.dSTD = paramDict['dSTD']
        self.lSTD = paramDict['lSTD']
        self.elDensity = paramDict['electronDensity']
        self.vDensity = paramDict['vacancyDensity']
        self.dDiameter = currDiameter
        self.average_ll = 0.5 #expect ligand length total between np to be 1 nm, so per np = 0.5 nm
        
        if stdMethod == "percent":
            self.np_d_std = self.dDiameter*self.dSTD
        elif stdMethod == "fixed":
            self.np_d_std = self.dSTD
            
        #define upper and lower bounds of size disorder
        self.np_ub = self.dDiameter + 2*self.average_ll
        self.np_lb = self.dDiameter - 2*self.average_ll

        #define upper and lower bounds of position disorder
        self.npl_ub = self.average_ll/2
        self.npl_lb = -self.average_ll/2
        
        #define where the first nanoparticle will sit
        a = self.dDiameter/2.0 #+ average_ll
        self.baseArray = [a,a,a]
        
        self.np_array = []
        
    def setLatticeAngles(self):
        #define lattice angles
        self.alpha = 99*math.pi/180
        self.beta = 99*math.pi/180
        self.gamma = 99*math.pi/180
        
    def setSpacingLattice(self):
        self.setLatticeAngles()
        
        #in the typical a,b,c representation, z will be in the b direction, x in a, and y in c
        self.sidelength = self.dDiameter + 2*self.average_ll
        self.a_1 = [0,0,self.sidelength]
        self.a_2 = [self.sidelength*math.sin(self.gamma),0,self.sidelength*math.cos(self.gamma)]
        
        x_3 = self.sidelength*(math.cos(self.beta)-math.cos(self.alpha)*math.cos(self.gamma))/math.sin(self.gamma)
        z_3 = self.sidelength*math.cos(self.alpha)
        y_3 = math.sqrt(self.sidelength**2-x_3**2-z_3**2)
        self.a_3 = [x_3,y_3,z_3]
        
        self.base_x = self.baseArray[0]
        self.base_z = self.baseArray[2]
        self.base_z_y = self.baseArray[2]
        
    def initiatePositions(self):
        self.curr_x = self.baseArray[0]
        self.curr_y = self.baseArray[1]
        self.curr_z = self.baseArray[2]

        self.cellx = self.baseArray[0]
        self.celly = self.baseArray[1]
        self.cellz = self.baseArray[2]
        
    def appendNP(self):
        #random size
        self.np_diameter = np.random.normal(self.dDiameter,self.np_d_std)
        while self.np_diameter < self.np_lb or self.np_diameter > self.np_ub:
            self.np_diameter = np.random.normal(self.dDiameter,self.np_d_std)
        #random location
        self.x_jit = np.random.normal(0,self.lSTD)
        self.y_jit = np.random.normal(0,self.lSTD)
        self.z_jit = np.random.normal(0,self.lSTD)
        self.total_jit = math.sqrt(self.x_jit**2 + self.y_jit**2 + self.z_jit**2)
        while self.total_jit < self.npl_lb or self.total_jit > self.npl_ub:
            self.x_jit = np.random.normal(0,self.lSTD)
            self.y_jit = np.random.normal(0,self.lSTD)
            self.z_jit = np.random.normal(0,self.lSTD)
            self.total_jit = math.sqrt(self.x_jit**2 + self.y_jit**2 + self.z_jit**2)
        #append nanoparticle
        self.np_array.append([self.curr_x + self.x_jit,self.curr_y+self.y_jit,self.curr_z+self.z_jit,self.np_diameter])
        
    def generateLattice(self):
        for y in range(1,self.yNum+1):
            for x in range(1,self.xNum+1):
                for z in range(1,self.zNum+1):
                    self.appendNP()
                    if y == 1 and x == 1 and z == self.zNum:
                        self.cellz = self.curr_z - self.cellz + self.dDiameter + self.z_jit
                        #due to triclinic nature, for volume of cell we want the length of the sides
                        self.cellz_edge = (self.zNum-1)*(self.sidelength) + self.dDiameter + self.z_jit
                    
                    self.curr_x += self.a_1[0]
                    self.curr_y += self.a_1[1]
                    self.curr_z += self.a_1[2]
                    
                self.base_z += self.a_2[2]
                self.curr_z = self.base_z
                
                if y == 1 and x == self.xNum:
                    self.cellx = self.curr_x - self.cellx + self.dDiameter + self.x_jit
                    #length of side in the x direction
                    self.cellx_edge = (self.xNum-1)*(self.sidelength) + self.dDiameter + self.x_jit
                    
                self.curr_x += self.a_2[0]
                self.curr_y += self.a_2[1]
                
            self.base_x += self.a_3[0]
            self.base_z_y += self.a_3[2]
            self.base_z = self.base_z_y
            
            self.curr_x = self.base_x
            self.curr_z = self.base_z
            
            self.curr_y += self.a_3[1]
            
        self.celly = self.curr_y - self.a_3[1] - self.celly + self.dDiameter + self.y_jit
        #length of side in the y direction
        self.celly_edge = (self.yNum-1)*(self.sidelength) + self.dDiameter + self.y_jit
        
    def calculateVolume(self):
        return self.cellx_edge*self.celly_edge*self.cellz_edge*math.sqrt(1-math.cos(self.alpha)**2 - math.cos(self.beta)**2-math.cos(self.gamma)**2 + 2*math.cos(self.alpha)*math.cos(self.beta)*math.cos(self.gamma))
        
    def createVacancies(self):
        numberVacancies = int(self.vDensity*len(self.np_array))
        for x in range(0,numberVacancies):
            randIndex = np.random.randint(0,len(self.np_array))
            #make sure that the nanoparticle being removed is not at either end in the z direction (will affect transport code)
            zValue = self.np_array[randIndex][2]
            while zValue < self.dDiameter or zValue > (self.cellz - self.dDiameter):
                randIndex = np.random.randint(0,len(self.np_array))
                zValue = self.np_array[randIndex][2]
            self.np_array.pop(randIndex)
        
    def returnLattice(self):
        self.setSpacingLattice()
        self.initiatePositions()
        self.generateLattice()
        self.createVacancies()
        #print("Cell x: %f Cell x edge: %f" %(self.cellx, self.cellx_edge))
        #print("Cell y: %f Cell y edge: %f" %(self.celly, self.celly_edge))
        #print("Cell z: %f Cell z edge: %f" %(self.cellz, self.cellz_edge))
        self.cellV = self.calculateVolume()
        #print("Cell volume: %f" %self.cellV)
        #self.elNum = self.elDensity*self.cellV
        self.cellsize_array = [self.cellx, self.celly, self.cellz]
        self.np_array_flipped = [[i[0], i[1], self.cellsize_array[2]-i[2], i[3]] for i in self.np_array] #flipped in the transport direction

        return self.np_array, self.np_array_flipped, self.cellsize_array,self.cellV
        
        
class cubic_lattice(triclinic_lattice):
    #def _init_(self,stdMethod,sizeRange,currDiameter,paramDict):
    #    super()._init_(stdMethod,sizeRange,currDiameter,paramDict)
    def setLatticeAngles(self):
        #define lattice angles
        self.alpha = 90*math.pi/180
        self.beta = 90*math.pi/180
        self.gamma = 90*math.pi/180
    
class cpc_lattice(triclinic_lattice):
    #def _init_(self,stdMethod,sizeRange,currDiameter,paramDict):
    #    super()._init_(stdMethod,sizeRange,currDiameter,paramDict)
    def setLatticeAngles(self):
        #define lattice angles
        self.alpha = 90*math.pi/180
        self.beta = 90*math.pi/180
        self.gamma = 90*math.pi/180
        
    def setLatticeSpacing(self):
        self.x_spacing = self.dDiameter + 2*self.average_ll
        self.y_spacing = layer_spacing = (dDiameter + 2*average_ll)*math.sqrt(3)/2
        self.z_spacing = self.dDiameter + 2*self.average_ll
        
    def generateLattice(self):
        for y in range(1,self.yNum+1):
            for x in range(1,self.xNum+1):
                for z in range(1,self.zNum+1):
                    self.appendNP()
                    if y == 2 and x == 1 and z == self.zNum:
                        cellz = curr_z - cellz + dDiameter + self.z_jit
                        self.cellz_edge = (self.zNum-1)*(self.sidelength) + self.dDiameter + self.z_jit
                    curr_z += self.z_spacing
                    
                self.curr_z = self.baseArray[2]
                if y == 2 and x == self.xNum:
                    self.cellx = self.curr_x - self.cellx + self.dDiameter + self.x_jit
                    self.cellx_edge = (self.xNum-1)*(self.sidelength) + self.dDiameter + self.x_jit
                curr_x += bottom_spacing
            if y%2 != 0:
                self.curr_x = self.baseArray[0] + self.x_spacing/2.0
                self.curr_z = self.baseArray[2] + self.z_spacing/2.0
            else:
                self.curr_x = self.baseArray[0]
                self.curr_z = self.baseArray[2]
                
            self.curr_y += self.y_spacing
            
        self.celly = self.curr_y - self.y_spacing - self.celly + self.dDiameter + self.y_jit  
        self.celly_edge = (self.yNum-1)*(self.sidelength) + -self.celly + self.dDiameter + self.y_jit

In [9]:
def createSample(method,stdMethod,sizeRange, currDiameter, paramDict):
    if method == "cubic":
        sample = cubic_lattice(stdMethod,sizeRange, currDiameter, paramDict)
    elif method == "cpc":
        sample = cpc_lattice(stdMethod,sizeRange, currDiameter, paramDict)
    elif method == "tric":
        sample = triclinic_lattice(stdMethod,sizeRange, currDiameter, paramDict)
    
    return sample.returnLattice()

In [10]:
stdMethod = "fixed"
#percent: percent of mean
#fixed: user set value

sizeRange = True
#true: will generate nanoparticles samples in a size range (for diameter)
#false: will generate nanoparticles at only one diameter
    
paramDict = getParams(stdMethod, sizeRange)
numberSamples = int(input("How many samples (must be even number)?: "))
npnumber = paramDict['npnumber']
dDiameter = paramDict['dDiameter']
elDensity = paramDict['electronDensity']

if sizeRange:
    currDiameter = dDiameter[0]
    while currDiameter <= dDiameter[1]:

        #generate samples, with inverted pairs
        sampleArray = []
        s = 0
        total_cellV = 0
        while s < (numberSamples/2):
            method = "cubic"
            #cubic = simple cubic
            #cpc = close packed cubic (body centered)
            #tric = triclinic, angles are defined inside the triclinic sample generation code

            np_array, np_array_flipped, cellsize_array, cellV = createSample(method,stdMethod,sizeRange, currDiameter, paramDict)
            sampleArray.append([np_array, cellsize_array])
            sampleArray.append([np_array_flipped, cellsize_array])
            total_cellV += cellV
            s += 1

        #print out electron numbers for each nanoparticle, will be slightly different for each sample, so average over volume
        average_cellV = 2*total_cellV/numberSamples
        elNumber = elDensity*average_cellV
        #print("NP Diameter: %f,  Number of electrons: %d" %(dDiameter, elNumber))
        print("NP Diameter: %f,  Number of electrons: %d" %(currDiameter, elNumber))
        
        #assign file path
        filePath = 'c:/Users/Davis/Research/data/nanoparticles/' + str(npnumber) + '_' + str(currDiameter) + 'nm/'
        if not os.path.exists(filePath):
            os.makedirs(filePath)
        
        #output samples
        i = 0
        while i < (numberSamples):
            filename = "nanoparticles" + str(i) + ".inp"
            fullpath = os.path.join(filePath, filename)

            np_array = sampleArray[i][0]
            cellsize_array = sampleArray[i][1]

            with open(fullpath,'w+') as f:
                f.write("BANDS\n")
                f.write("8 8\n") #degeneracy , always 8 8 for PbSe
                f.write("NANOPARTICLES\n")
                f.write("cell(1), %f\n" %cellsize_array[0])#X  center-center distance leftmost-rightmost + desired diameter
                f.write("cell(2), %f\n" %cellsize_array[1])#Y  Defines height (layers)
                f.write("cell(3), %f\n" %cellsize_array[2])#Z  Transport direction

                for j in range(0,len(np_array)):
                    x = str(np_array[j][0])
                    y = str(np_array[j][1])
                    z = str(np_array[j][2])
                    d = str(np_array[j][3])
                    f.write(x + ", " + y + ", " + z + ", " + d + "\n")
                    
            i += 1
        
        #np diameter increment
        currDiameter += 0.5
else:
    filePath = 'c:/Users/Davis/Research/data/nanoparticles/' + str(npnumber) + '_' + str(dDiameter) + 'nm/'
    if not os.path.exists(filePath):
        os.makedirs(filePath)

    #generate samples, with inverted pairs
    sampleArray = []
    s = 0
    total_cellV = 0
    while s < (numberSamples/2):
        method = "cubic"
        #cubic = simple cubic
        #cpc = close packed cubic (body centered)
        #tric = triclinic, angles are defined above

        np_array, np_array_flipped, cellsize_array, cellV = createSample(method,stdMethod,sizeRange, dDiameter, paramDict)
        sampleArray.append([np_array, cellsize_array])
        sampleArray.append([np_array_flipped, cellsize_array])
        
        total_cellV += cellV

        s += 1

    #print out electron numbers for each nanoparticle, will be slightly different for each sample, so average over volume
    average_cellV = 2*total_cellV/numberSamples
    elNumber = elDensity*average_cellV
    print("NP Diameter: %f,  Number of electrons: %d" %(dDiameter, elNumber))
    
    #output samples
    i = 0
    while i < (numberSamples):
        filename = "nanoparticles" + str(i) + ".inp"
        fullpath = os.path.join(filePath, filename)

        np_array = sampleArray[i][0]
        cellsize_array = sampleArray[i][1]

        with open(fullpath,'w+') as f:
            f.write("BANDS\n")
            f.write("8 8\n") #degeneracy , always 8 8 for PbSe
            f.write("NANOPARTICLES\n")
            f.write("cell(1), %f\n" %cellsize_array[0])#X  center-center distance leftmost-rightmost + desired diameter
            f.write("cell(2), %f\n" %cellsize_array[1])#Y  Defines height (layers)
            f.write("cell(3), %f\n" %cellsize_array[2])#Z  Transport direction

            for j in range(0,len(np_array)):
                x = str(np_array[j][0])
                y = str(np_array[j][1])
                z = str(np_array[j][2])
                d = str(np_array[j][3])
                f.write(x + ", " + y + ", " + z + ", " + d + "\n")
                
        i += 1

Number x nanoparticles: 10
Number y nanoparticles: 2
Number z nanoparticles: 20
Desired Diameter range (in nm), use ' ' to separate: 3 8
Density of electrons (#/nm^3): .002
Nanoparticle Size Standard Deviation (in nm): .3
Nanoparticle Location Standard Deviation (in nm): 0
Percent Vacancies (in the QDxV(1-x) style): 0
How many samples (must be even number)?: 100
NP Diameter: 3.000000,  Number of electrons: 43
NP Diameter: 3.500000,  Number of electrons: 62
NP Diameter: 4.000000,  Number of electrons: 87
NP Diameter: 4.500000,  Number of electrons: 117
NP Diameter: 5.000000,  Number of electrons: 154
NP Diameter: 5.500000,  Number of electrons: 198
NP Diameter: 6.000000,  Number of electrons: 249
NP Diameter: 6.500000,  Number of electrons: 308
NP Diameter: 7.000000,  Number of electrons: 376
NP Diameter: 7.500000,  Number of electrons: 454
NP Diameter: 8.000000,  Number of electrons: 541
